In [1]:
import os
import pandas as pd
import DataBase
import ShockFunction
import nesting_trees as nt
import abatement_functions as af
import DB2Gams

In [2]:
data_folder = os.getcwd()+'\\Make_data\\Example2'

# **Example 2**

The general setup is as in example 1, however, the focus here is more on the abatement model than on how to run things from Python.

As in example 1 the setup still involves:
* A set of energy-services $(\mathcal{S}_{es})$,
* A set of fuels in production $(\mathcal{S}_{f})$,
* For each $es\in\mathcal{S}_{es}$, the energy-service can be divided into a set of energy-service-components $(\mathcal{S}_{esc}^{es})$. The set of all energy-service-components is denoted $\mathcal{S}_{esc}$. 
* All sets of energy-service components $(\mathcal{S}_{esc}^{es})$ can be divided into a baseline component $(es^0)$ and technology components $(\tilde{\mathcal{S}}_{esc}^{es})$. (*The naming convention is that baseline components are defined as the energy-service $+\_0$)*
* The technology components are all defined by a set of *technology-firms* $(\mathcal{T})$. The technologies uses a combination of fuels and capital to produce one or more components.

## **1: The Setup**

### **1.1: Read in data**

#### **1.1.1: Final goods sector**

Uses a combination of capital (K) and energy-services $(E_1,...,E_{n_e})$ to produce an output $Y$. The nesting structure can be chosen arbitrarily using nesting trees as in example 1.

Example 1 illustrated how to build nesting trees. An alternative to this is simply to let data define the tree:

In [3]:
trees = {}
read_type = {'1dvars': ['sigma','q'], 'vars_panel': {'mu': 2}}
trees['FG'] = nt.nt_base_v2(from_data=True,data_path=data_folder+'\\FG.xlsx',name='FG')
trees['FG'].database.read_from_excel(data_folder+'\\FG.xlsx',{**read_type, 'maps_panel': ['n2X']})
trees['FG'].run_all_v2()

#### **1.1.2: Energy-service sector**

In [4]:
trees['ES'] = nt.tree_from_data(data_folder+'\\ES.xlsx',name='ES')
trees['ES'].run_all()
trees['ES'].database.read_from_excel(data_folder+'\\ES.xlsx',read_type)

#### **1.1.3: Energy services, base supply**

Uses a combination of capital and fuels to produce energy-services using CES.

In [5]:
trees['ES_base'] = nt.nt_base_v2(from_data=True,data_path=data_folder+'\\ES_base.xlsx',name='ES_base')
trees['ES_base'].database.read_from_excel(data_folder+'\\ES_base.xlsx',{**read_type, 'maps_panel': ['n2X']})
trees['ES_base'].run_all_v2()

#### **1.1.4: Energy-service-components sector**

In [6]:
trees['ESC'] = nt.tree_from_data(data_folder+'\\ESC.xlsx',name='ESC')
trees['ESC'].run_all()
trees['ESC'].database.read_from_excel(data_folder+'\\ESC.xlsx',read_type)

#### **1.1.5: technology sector**

In [7]:
trees['T'] = nt.nt_CET_v2(from_data=True,data_path=data_folder+'\\T.xlsx',name='T')
trees['T'].database.read_from_excel(data_folder+'\\T.xlsx',{**read_type, 'maps_panel': ['n2X']})
trees['T'].run_all_v2()

### **1.2: Technologies and fuels**

A set of technology firms $(\mathcal{T})$ produces energy-service-components - all of them using fuels and capital. In the model that entails that fuels and capital enters with one price, but many different quantities (one for each firm applying them). To handle this:
* Define quantity-variables with the naming convention 'fueltype'+'$\_$firmtype'.
* Define price-variables over 'fueltype' (not entire 'fueltype$\_$firmtype'),
* Define a mapping from 'fueltype_$\_$firmtype'$ to 'fueltype'.

To handle that quantities and prices are defined over overlapping, but different sets, we define a mapping from elements in the set prices are defined over, to its counterpart in quantities. For many elements this mapping will be neutral ('x' to 'x'), however, for some it will not be.

## **2: Write to gms**

We follow the same procedure as in Example1.ipynb. Here we have 3 different versions of the abatement model:
* 'am_base': Standard model with arbitrarily nested input structure.
* 'am_base_v2': 'am_base' with the price-vector defined over 'fueltype' instead of 'fueltype-firmtype' combinations.
* 'am_CET_v2': Arbitraily nested input/output structure, with the price-vector defined over 'fueltype' instead of 'fueltype-firmtype' combinations.

In [8]:
repos = {key: os.getcwd()+'\\gms\\Example2\\'+key for key in trees}
model_settings = {}
model_settings['FG'] = af.am_base_v2(trees['FG'])
model_settings['ES'] = af.am_base(trees['ES'])
model_settings['ESC'] = af.am_base(trees['ESC'])
model_settings['ES_base'] = af.am_base_v2(trees['ES_base'])
model_settings['T'] = af.am_CET_v2(trees['T'])

Write the gams code for the models with the same types of nesting (CES,normalized CES and MNL) as in Example1.ipynb:

In [9]:
model_settings['FG'].run_abatement_model(repo=repos['FG'],type_='CES_v2',export_settings=True)
model_settings['ES'].run_abatement_model(repo=repos['ES'],type_='normalized_CES',export_settings=True)
model_settings['ESC'].run_abatement_model(repo=repos['ESC'],type_='normalized_CES',export_settings=True)
model_settings['ES_base'].run_abatement_model(repo=repos['ES_base'],type_='CES_v2',export_settings=True)
model_settings['T'].run_abatement_model(repo=repos['T'],type_in='CES_v2',type_out='MNL',export_settings=True,add_aggregates=True)

Note that for sectors where prices/quantities are defined over different sets, we use the type 'CES_v2' instead of just CES. Similarly, in one of the sectors we add the argument 'add_aggregates=True': This adds a simple block of equations that sums of the various fuel- and capital types into aggregate ones. As we are interested in the combined model, we only need one of these blocks to be included, to get the aggregates in the end.

## **3: Run models separately**

In [10]:
workfolder = os.getcwd()+'\\workfolder'
models = {key: DB2Gams.gams_model(workfolder) for key in model_settings}
[models[key].run(model_settings[key].model) for key in model_settings];

### **4: Combine models**

Create list of *gams_settings* objects:

In [11]:
list_of_settings = [model_settings[key].model for key in model_settings]

Merge settings:

In [12]:
merged_settings = DB2Gams.mgs.merge(list_of_settings)

Run combined model with a checkpoint:

In [13]:
c_model= DB2Gams.gams_model(workfolder)
cp = c_model.ws.add_checkpoint() # create empty checkpoint
c_model.run(merged_settings,options_run={'checkpoint': cp}) # run model and store in checkpoint cp

## **5: A wee bit of analysis**

In Example1.ipynb the price on one type of fuel was gradually increased, by looping over the model from Python. Here, we write the loop directly to gams, and run the loop there.

### **5.1: Using the AddShock class**

#### **5.1.1: Create a database with the shocks we want to loop over:**

In [14]:
shocks = DataBase.py_db(name='shock')

Note that we can loop sparsely by defining a multiindex to loop over, for instance 'loop3' here which is a combination of two 'fundamental' loops:

In [15]:
shocks['l1'] = pd.Index(range(1,3),name='l1').astype(str)
shocks['l2'] = pd.Index(range(1,3),name='l2').astype(str)
shocks['loop3'] = pd.MultiIndex.from_tuples([('1','1'),('2','2')], names=['l1','l2'])

Add parameters with the values we want to loop through:

In [16]:
shocks['p1'] = pd.Series([1,2],index=shocks['l1'],name='p1')
shocks['p2'] = pd.Series([1,1.5],index=shocks['l2'],name='p2')
shocks['p1'].attrs['type'] = 'parameter'
shocks['p2'].attrs['type'] = 'parameter'

Export to workfolder:

In [17]:
shocks.merge_internal()
shocks.db_other.export(workfolder+'\\'+shocks.name+'.gdx')

#### **5.1.2: Specify what happens in the loop**

We use the class AddShocks to define what happens in the loop. To initialize the class we provide: (1) Name of the relevant model that is solved in the loop, (2) the database with relevant shocks, and (3) the name of the 'sparse loop', i.e. the one encompassing all relevant elements we wish to loop over. In this case it was the multiindex 'loop3':

In [18]:
s1 = ShockFunction.AddShocks('model_name',shocks,'loop3')

We perform a shock of the type 'UpdateExoVarsAndSolve', that is where exogenous values are updated in each iteration of the loop, and the model is then resolved:

In [19]:
s1.UpdateExoVarsAndSolve(c_model)

Let the price for 'F1' follow the shock 'p1', and the price for 'F2' follow 'p2'. This is added by adding: The variable name, the corresponding parameter that we wish to update with, and finally conditions on the solution:

In [20]:
s1.UEVAS_adjVar('p','p1',conditions='sameAs(n,"F1")') # adjust the variable p with the value from parameter p1, only for p[n] when n = F1.
s1.UEVAS_adjVar('p','p2',conditions='sameAs(n,"F2")') # adjust the variable p with the value from parameter p1, only for p[n] when n = F1.

Let us extract the solution for quantities (q) for each realization, but only for the subset n2nn_agg (capital and fuel aggregtaes). Furhtermore, let us define a new result for all combinations of the loops ('loop3' contains all combination):

In [21]:
s1.UEVAS_var2sol('q','loop3',conditions='n2nn_agg[n]')

### **5.2: Running the shock**

To run the shock, we need to provide: (1) The name of the shock-data, (2) The shock-code from the AddShocks class.

In [22]:
c_model.opt.defines['shock'] = shocks.name+'.gdx'

Write the shock-code to a gmy file:

In [23]:
s1.UEVAS_2gmy(workfolder+'\\'+s1.shock_gm.database.name)

Write from file:

In [24]:
s1.gmy

'C:\\Users\\sxj477\\Documents\\GitHub\\Abatement_project\\InputDisplacingAbatement\\workfolder\\shock.gmy'

In [25]:
c_model.job = c_model.ws.add_job_from_file(s1.gms,cp) 
c_model.run(run_from_job=True)

In [ ]:
test = Precompiler(fil)